Make that we are running the correct version of TensorFlow first

In [2]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [3]:
import sys

assert sys.version_info >= (3, 6) # Python ≥3.6 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

# Train basic model on the generated emnist-lines dataset

In [146]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
# To be able to write code as if this notebook was one level up in files tree structure.
sys.path.append('..')

from pathlib import Path

import tensorflow as tf
from tensorflow.python.ops import array_ops
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.models import Model as KerasModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

import pandas as pd
from PIL import Image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
from recognizer.datasets import Dataset
from recognizer.datasets import EmnistDataset
from recognizer.networks import lenet
from recognizer.networks import simple
from recognizer.networks import NetworkInput

Hyperparameters

In [6]:
batch_size = 256
epochs = 16

## Load characters mapping

In [126]:
import json

with open(Dataset.raw_data_path()/"emnist"/"emnist_essentials.json") as json_file:  
    mapping = json.load(json_file)["mapping"]
    mapping = {m[0]: m[1] for m in mapping}
    mapping[62] = ' '
    mapping[63] = '_'
    
mapping_reversed = {v: k for k, v in mapping.items()}

## Build the new model

In [21]:
char_image_height = 28
char_image_width = 28
max_length = 34
num_classes = 64
image_height = char_image_height
image_width = char_image_width * max_length

input_shape = (image_height, image_width)
output_shape = (max_length, num_classes)

window_width: float = 16
window_stride: float = 8

In [132]:
model = Sequential()

# model.add(layers.Reshape((image_height, image_width, 1), input_shape=input_shape))
model.add(layers.Input((image_height, image_width, 1), name="input"))

# # Normalize the image
# model.add(layers.Lambda(lambda x: norm(x, network_input.mean, network_input.std), input_shape=input_shape, output_shape=input_shape))

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.2))
# (image_height // 2 - 2, image_width // 2 - 2, 64)

# -2 because we use the default padding which is 'valid' not 'same'.
# /2 because we did maxPooling with pool size 2, this halved the output size of the previous conv layer.
new_height = image_height // 2 - 2
new_width = image_width // 2 - 2
# Let us also scale the desired window width and stride accordingly.
new_window_width = window_width // 2 - 2
new_window_stride = window_stride // 2
model.add(layers.Conv2D(128, kernel_size=(new_height, new_window_width), strides=(1, new_window_stride), activation='relu'))
model.add(layers.Dropout(0.2))
# (1, num_windows, 128)

num_windows = int((new_width - new_window_width) / new_window_stride) + 1
model.add(layers.Reshape((num_windows, 128, 1)))
# (num_windows, 128, 1)

width = int(num_windows / max_length)
model.add(layers.Conv2D(num_classes, kernel_size=(width, 128), strides=(width, 1), activation='softmax'))
# (image_width / width, 1, num_classes)

model.add(layers.Lambda(lambda x: tf.squeeze(x, 2)))
# (max_length, num_classes)

# Since we floor'd the calculation of width, we might have too many items in the sequence. Take only output_length.
model.add(layers.Lambda(lambda x: x[:, :max_length, :]))

In [133]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 26, 950, 32)       320       
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 24, 948, 64)       18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 12, 474, 64)       0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 12, 474, 64)       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 1, 118, 128)       589952    
_________________________________________________________________
dropout_26 (Dropout)         (None, 1, 118, 128)       0         
_________________________________________________________________
reshape_13 (Reshape)         (None, 118, 128, 1)     

## Look at the data

In [62]:
df = pd.read_csv(Dataset.processed_data_path()/"emnist_lines"/"data.csv")

output_length = len(df)

In [63]:
df.head()

,image,sentence
0,0.png,Whats wrong at state _____________
1,1.png,The transducer itself moves the __
2,2.png,There is an ancient and venerable
3,3.png,They destroyed a trading house ___
4,4.png,Alec waited a moment on guard ___


In [64]:
image = Image.open(Dataset.processed_data_path()/"emnist_lines"/"0.png")
image

In [65]:
np.array(image).shape

(28, 952)

## Try it before training

In [183]:
image_numpy = np.array(image).astype(np.float32)
test_batch = image_numpy.reshape(1, image_height, image_width, 1)

preds = model(test_batch)
preds.shape

TensorShape([1, 34, 64])

In [184]:
image

In [185]:
for char_pred in preds[0]:
    print(mapping[np.argmax(char_pred)], end='')

ehats _ in  __ s_ateo n___________

## Train it

In [180]:
def generate_train_data():
    while True:
        for index, row in df.iterrows():
            batch_x = []
            batch_y = []

            for i in range(batch_size):
                image = Image.open(Dataset.processed_data_path()/"emnist_lines"/row['image'])
                x = np.array(image).astype(np.float32).reshape(image_height, image_width, 1)
                batch_x.append(x)
                
                y = [mapping_reversed[char] for char in row['sentence']]
                y = to_categorical(y, num_classes).astype(np.int)
                batch_y.append(y)

            yield np.array(batch_x), np.array(batch_y)

Check progress in TensorBoard by running `tensorboard --logdir=logs/lines_all_conv` in terminal

In [182]:
callbacks = [
#     keras.callbacks.EarlyStopping(
#         # Stop training when `accuracy` is no longer improving
#         monitor='accuracy',
#         # "no longer improving" being defined as "no better than 1e-2 less"
#         min_delta=1e-2,
#         # "no longer improving" being further defined as "for at least 2 epochs"
#         patience=2,
#         verbose=1),
    keras.callbacks.TensorBoard(log_dir='../logs/lines_all_conv')
]

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
model.fit_generator(generate_train_data(),
                    steps_per_epoch=len(df) // batch_size,
                    epochs=epochs,
                    callbacks=callbacks)

Epoch 1/16
39/39 [==============================] - 291s 7s/step - loss: 2.3050 - accuracy: 0.3516
Epoch 2/16
26/39 [===================>..........] - ETA: 1:36 - loss: 2.6337 - accuracy: 0.2485

KeyboardInterrupt: 